In [2]:
import pandas as pd

df = pd.read_csv('train_test_data.csv')
df = df.groupby('patientunitstayid').apply(lambda x: x[x['observationoffset'] <= 24]).reset_index(drop=True)
df.head()

,patientunitstayid,observationoffset,heartrate,temperature,paO2,FiO2_x,FiO2_y,Glasgow score,BP,BUN,WBC x 1000,bicarbonate,sodium,potassium,total bilirubin,actualicumortality
0,141515.0,0,86.823529,36.900429,108.5,69.034181,69.034181,6.689885,87.272727,75.049159,11.7,20.589048,133.145147,3.600000,1.2,1
1,141515.0,1,90.222222,36.958959,283.0,100.000000,100.000000,6.578818,111.000000,74.894458,11.7,21.000000,132.000000,3.600000,1.2,1
2,141515.0,2,91.600000,36.584615,108.5,68.208171,68.208171,6.462798,120.800000,74.731208,11.7,20.553918,133.426232,3.704932,1.2,1
3,141515.0,3,87.875000,38.775000,55.0,50.000000,50.000000,6.342544,111.062500,74.559406,11.7,20.540583,133.569442,3.876114,1.2,1
4,141515.0,4,85.875000,38.362500,71.0,70.000000,70.000000,6.218848,115.333333,74.379056,11.7,20.530217,133.714358,4.000000,1.2,1


In [ ]:
# select patient with actualicumortality = 1
patient_alive = df[df['actualicumortality'] == 1]
alive_id = patient_alive['patientunitstayid'].unique()

# select patient with actualicumortality = 0
patient_dead = df[df['actualicumortality'] == 0]
dead_id = patient_dead['patientunitstayid'].unique()
print(len(alive_id))
print(len(dead_id))

# patient_alive_sample = patient_alive.sample(n=patient_dead.shape[0])
# data_set = pd.concat([patient_alive_sample, patient_dead])
# print(data_set.shape)

7526
1414


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# 确保随机性
np.random.shuffle(dead_id)
np.random.shuffle(alive_id)

# 从alive_id中随机抽取与dead_id相同数量的ID
alive_id = alive_id[:len(dead_id)]

# 确定训练集和测试集的大小
train_size = 0.8  # 训练集占 80%
test_size = 0.2   # 测试集占 20%

# 分别对死亡和生存的ID进行分割，确保训练集和测试集的平衡
dead_train, dead_test = train_test_split(dead_id, test_size=test_size)
alive_train, alive_test = train_test_split(alive_id, test_size=test_size)

# 将训练集的死亡和生存ID合并，并从原始数据集中提取对应记录
train_ids = np.concatenate((dead_train, alive_train))
train_data = df[df['patientunitstayid'].isin(train_ids)]

# 将测试集的死亡和生存ID合并，并从原始数据集中提取对应记录
test_ids = np.concatenate((dead_test, alive_test))
test_data = df[df['patientunitstayid'].isin(test_ids)]

# 打印训练集和测试集的大小
print(f"Training data size: {len(train_data)}")
print(f"Testing data size: {len(test_data)}")

Training data size: 55064
Testing data size: 13860


In [ ]:
X_train = train_data.drop(['actualicumortality', 'patientunitstayid'], axis=1)
y_train = train_data['actualicumortality']
X_test = test_data.drop(['actualicumortality', 'patientunitstayid'], axis=1)
y_test = test_data['actualicumortality']

# normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# # split data into train and test
# from sklearn.model_selection import train_test_split

# # X = data_set.drop(['actualicumortality', 'patientunitstayid', 'observationoffset'], axis=1)
# X = data_set.drop(['actualicumortality', 'patientunitstayid'], axis=1)
# y = data_set['actualicumortality']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [3]:
# train logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

logistic_model = LogisticRegression(max_iter=10000, n_jobs=-1)
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'confusion_matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'classification_report: \n{classification_report(y_test, y_pred)}')

# show the coefficients
print(logistic_model.coef_)

from sklearn.metrics import roc_curve, roc_auc_score



NameError: name 'X_train' is not defined

In [ ]:
# train random forest model
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_jobs=-1, n_estimators=200, max_depth=10)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'confusion_matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'classification_report: \n{classification_report(y_test, y_pred)}')

accuracy: 71.21%
confusion_matrix: 
[[4873 1954]
 [2036 4997]]
classification_report: 
              precision    recall  f1-score   support

           0       0.71      0.71      0.71      6827
           1       0.72      0.71      0.71      7033

    accuracy                           0.71     13860
   macro avg       0.71      0.71      0.71     13860
weighted avg       0.71      0.71      0.71     13860



In [ ]:
# train xgboost model
from xgboost import XGBClassifier

xgboost_model = XGBClassifier(n_jobs=-1)
xgboost_model.fit(X_train, y_train)
y_pred = xgboost_model.predict(X_test)

print(f'accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'confusion_matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'classification_report: \n{classification_report(y_test, y_pred)}')

accuracy: 69.52%
confusion_matrix: 
[[4660 2167]
 [2057 4976]]
classification_report: 
              precision    recall  f1-score   support

           0       0.69      0.68      0.69      6827
           1       0.70      0.71      0.70      7033

    accuracy                           0.70     13860
   macro avg       0.70      0.70      0.70     13860
weighted avg       0.70      0.70      0.70     13860

